In [ ]:
import os
import sys
from collections import defaultdict
from pathlib import Path
from typing import Dict, List, Literal, Optional, Tuple, Union
import logging
import IPython
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import display

notebook_path = Path(IPython.extract_module_locals()[1]["__vsc_ipynb_file__"])
project_dir = notebook_path.parent.parent
sys.path.append(str(project_dir))
import src.utils.custom_log as custom_log
from src.utils.Csv import Csv
from src.utils.set_rcparams import set_rcparams

os.chdir(project_dir)
LOG = logging.getLogger(__name__)
custom_log.init_logger(log_lvl=logging.INFO)
LOG.info("Log initialized")

In [ ]:
set_rcparams()
PIC_DIR = Path("reports/figures/validity_chain") / notebook_path.stem
PIC_DIR.mkdir(parents=True, exist_ok=True)
PAGE_WIDTH = 448.13095 / 72 -0.2

# Get Data

In [ ]:
def get_data(b_path: Path, cfc: Literal["A", "B", "C", "D", "X"]) -> pd.DataFrame:
    case_injs = []
    for inj_data in b_path.rglob("injury_criteria.csv.zip"):
        LOG.info("Process %s", inj_data.parent.stem)
        db = Csv(csv_path=inj_data, compress=True).read()
        vals = db.loc[cfc]
        vals.name = int(inj_data.parent.stem[1:])
        case_injs.append(vals)

    db = pd.concat(case_injs, axis=1).T
    db.index.name = "SID"
    db.rename(columns={"Chest_Deflection": "Chest_Defl.", "Femur_Fz_Max": "Femur_Fz"}, inplace=True)

    return db


DB: pd.DataFrame = get_data(
    b_path=Path("/mnt") / "q" / "Honda_Accord_2014_Sled_with_HIII_Rigid_Seat_SpeedOpt_BigDOE" / "doe_para_infl_20230921_163501",
    cfc="D",
)


DB

In [ ]:
LIMS = {
        "Head_a3ms": [80, "g"],
        "Neck_Nij": [1, ""],
        "Chest_a3ms": [60, "g"],
        "Chest_Defl.": [63, "mm"],
        "Femur_Fz": [10, "kN"],
    }

LIM_RENAMER = {
    "Head_a3ms": "Head HAC$_3$",
    "Neck_Nij": "Neck Nij",
    "Chest_a3ms": "Chest CAC$_3$",
    "Chest_Defl.": "Chest CDC",
    "Femur_Fz": "Femur FCC",
}

DB_SCAL = pd.DataFrame(index=DB.index)
for lim in LIMS.keys():
    DB_SCAL[lim] = DB[lim] / LIMS[lim][0]

DB_SCAL


In [ ]:
DOE = pd.read_excel(
    Path("/mnt")
    / "q"
    / "Honda_Accord_2014_Sled_with_HIII_Rigid_Seat_SpeedOpt_BigDOE"
    / "doe_para_infl_20230921_163501"
    / "doe.xlsx",
    index_col=0,
)
DOE["SLL"] /= 1000
DOE

In [6]:
PARAS = {
    "Pulse_X_Scale": "PSCAL [1]",
    "Pulse_Angle": "ALPHA [deg]",
    "PAB_M_Scal": "PABSCAL [1]",
    "PAB_Vent_T": "PABVENT [s]",
    "SLL": "SLL [kN]",
}
# drop initial velocity as motion is applied by acceleration curve --> initial velocity has no effect :(

# Evaluate

In [ ]:
def plot_influences():
    mids = [1, 14, 27]

    fig, ax = plt.subplots(
        ncols=len(LIMS.keys()),
        nrows=len(PARAS),
        constrained_layout=True,
        figsize=(PAGE_WIDTH, 1.2 * PAGE_WIDTH),
    )

    for m, perc in enumerate(DOE["PERC"].unique()):
        doe_perc = DOE[DOE["PERC"].eq(perc)]
        for i, para in enumerate(PARAS):
            idxs = [doe_perc[para].idxmin(), mids[m], doe_perc[para].idxmax()]
            ax[i, 0].set_ylabel(PARAS[para])

            for j, inj_val in enumerate(LIMS.keys()):
                unit = str(LIMS[inj_val][0]) + (("\u2009" + LIMS[inj_val][1]) if LIMS[inj_val][1] else "")
                ax[0, j].set_xlabel(f"{inj_val.replace('_', ' ')} [{unit}]")
                ax[0, j].xaxis.set_label_position("top")
                ax[i, j].plot(
                    doe_perc.loc[idxs, para], DB_SCAL.loc[idxs, inj_val], label=f"{perc:02.0f}", marker="|"
                )

                ax[i, j].set_xticks(doe_perc.loc[idxs, para])
                ax[i, j].grid()
                ax[i, j].set_ylim([0.1, 1.2])
                if j != len(LIMS.keys()) - 1:
                    ax[i, j].set_yticklabels([])
            ax[i, -1].yaxis.tick_right()
    ax[0, 1].legend()

    for suffix in ("png", "pdf"):
        fig.savefig(PIC_DIR / f"factor_influences.{suffix}")


plot_influences()

In [ ]:
def get_effects() -> Dict[str, pd.DataFrame]:
    mids = [1, 14, 27]

    all_effects = {}
    for m, perc in enumerate(DOE["PERC"].unique()):
        doe_perc = DOE[DOE["PERC"].eq(perc)]
        effects = defaultdict(dict)
        for i, para in enumerate(PARAS):
            idxs = [doe_perc[para].idxmin(), mids[m], doe_perc[para].idxmax()]

            for j, inj_val in enumerate(LIMS.keys()):
                LOG.debug("Dummy %s, Factor %s, Injury %s", perc, para, inj_val)

                m1 = np.abs(DB_SCAL.loc[idxs[1], inj_val]-DB_SCAL.loc[idxs[0], inj_val])
                m2 = np.abs(DB_SCAL.loc[idxs[2], inj_val]-DB_SCAL.loc[idxs[1], inj_val])
                m3 = np.abs(DB_SCAL.loc[idxs[2], inj_val]-DB_SCAL.loc[idxs[0], inj_val]) / 2
                effects[PARAS[para].split()[0]][inj_val.replace("_", " ")] = max(m1, m2, m3)
        all_effects[perc] = pd.DataFrame(effects).T

    return all_effects


EFFECTS = get_effects()
EFFECTS

In [ ]:
def plot_influences_2():
    mids = [1, 14, 27]

    fig, ax = plt.subplots(
        ncols=len(LIMS.keys()),
        nrows=len(PARAS),
        layout="constrained",
        sharex=True,
        sharey=True,
    )

    for m, perc in enumerate(DOE["PERC"].unique()):
        doe_perc = DOE[DOE["PERC"].eq(perc)]
        for i, para in enumerate(PARAS):
            idxs = [doe_perc[para].idxmin(), mids[m], doe_perc[para].idxmax()]
            ax[i,0].annotate(PARAS[para].split()[0], xy=(-0.65, 0.5), xycoords="axes fraction", ha="center", va="center", rotation=90)
            #ax[i, 0].set_ylabel(PARAS[para].split()[0])

            for j, inj_val in enumerate(LIMS.keys()):
                unit = str(LIMS[inj_val][0]) + ((" " + LIMS[inj_val][1]) if LIMS[inj_val][1] else "")
                # ax[0, j].set_xlabel(f"{LIM_RENAMER[inj_val]}\n\nNormalized by {unit}")
                ax[0, j].set_title(LIM_RENAMER[inj_val].split()[0])
                # ax[0, j].xaxis.set_label_position("top")
                ax[i,j].annotate(f"Normalized {unit}", xy=(0.5, 0.0), xycoords="axes fraction", ha="center", va="bottom", fontsize=7, weight="ultralight")

                ax[i, j].plot(
                    [-1, 0, 1],
                    DB_SCAL.loc[idxs, inj_val],
                    label=f"{EFFECTS[perc].loc[PARAS[para].split()[0], inj_val.replace('_', ' ')]:.2f}",
                    marker="|",
                )

                # ax[i, j].set_xticks(doe_perc.loc[idxs, para])
                ax[i, j].grid()
                ax[i, j].set_ylim([0, 1.4])
                ax[i, j].set_xlim([-1, 1])
                ax[i, j].legend()

                #ax[i, j].set_xlabel(PARAS[para].split()[0])
                ax[i, j].set_ylabel(LIM_RENAMER[inj_val].split()[1])

                #if j !=0:
                #    ax[i, j].set_yticklabels([])
                if i == len(PARAS) - 1:
                    ax[i, j].set_xlabel("Unit Factor")
            #ax[i, -1].yaxis.tick_right()

    #fig.align_ylabels(ax)
    fig.set_figheight(1.3 * PAGE_WIDTH)
    fig.set_figwidth(PAGE_WIDTH)
    for suffix in ("pdf",):
        fig.savefig(PIC_DIR / f"factor_influences2.{suffix}")


plot_influences_2()

In [ ]:
def plot_effects():
    width = 448.13095 / 72
    fig, ax = plt.subplots(ncols=len(EFFECTS.keys()), sharex="all", sharey="all", figsize=(width, 0.5*width))
    for i, perc in enumerate(EFFECTS.keys()):
        sns.heatmap(EFFECTS[perc], ax=ax[i], cmap="Blues", square=True, annot=True, linewidths=0.1, cbar=False, fmt=".2f", vmin=0, vmax=0.2)
        ax[i].set_title(f"{perc}th Percentile")

    for suffix in ("png", "pdf"):
        fig.savefig(PIC_DIR / f"factor_effects.{suffix}")

plot_effects()

In [ ]:
sns.pairplot(DOE)